In [1]:
import os
from zoneinfo import reset_tzpath

import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader

load_dotenv()

/Users/ashiq/PycharmProjects/AIAgentDevelopment/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
reader = PdfReader("me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
linkedin

'\xa0 \xa0\nContact\n+8801607400180  (Home)\ngaziashiq.dev@gmail.com\nwww.linkedin.com/in/gaziashiq\n(LinkedIn)\nTop Skills\nFastAPI\nJSON\nCRUD\nCertifications\nJAVA under NASSCOM IT-\nITES Sector Skill Council (SSC)\nCertification\nIntroduction to Back-End\nDevelopment\nVersion Control\nProgramming in Python\nIntroduction to PythonGazi Ashiq\nBackend Developer (Python | Django | TypeScript)\nDhaka, Bangladesh\nSummary\nSoftware developer who works autonomously and strives to come\nup with solutions that are both viable and easy to maintain. I bring\ninto my work a penchant for improving processes, a lifelong learning\npursuit, and a desire to strengthen communities.\n- Creative, analytical and quick self-learner\n- Strong debugging and independent problem solving skills\n- Writing effective, scalable code\n- Developing back-end components to improve responsiveness and\noverall performance\nFeel free to email me: GaziAshiq.Dev@gmail.com\nEducation\nDaffodil International University-DI

In [5]:
with open("me/about.txt", "r") as file:
    summary = file.read()

In [6]:
summary

'Software developer who works autonomously and strives to come up with solutions that are both viable and easy to maintain. I bring into my work a penchant for improving processes, a lifelong learning pursuit, and a desire to strengthen communities.\n\n- Creative, analytical and quick self-learner\n- Strong debugging and independent problem solving skills\n- Writing effective, scalable code\n- Developing back-end components to improve responsiveness and overall performance\n\nFeel free to email me: GaziAshiq.Dev@gmail.comSoftware'

In [7]:
name = "Gazi Ashiq"

In [8]:
system_prompt = f"""You are acting as {name}. You are answering questions based on your LinkedIn profile and a summary of your professional background.
Your LinkedIn profile is as follows:
{linkedin}
Your summary is as follows:
{summary}
You are a professional in the field of AI and software engineering, with a focus on building AI
applications and web development. You have experience in various programming languages and frameworks, and you are passionate about creating innovative solutions.
You are friendly, professional, and concise in your responses. Always provide accurate information based on the
provided LinkedIn profile and summary. If you don't know the answer, say "I don't know" instead of making up an answer.
"""

In [9]:
def chatbot(message, history):
    if not message.strip():
        return "", history

    response = openai_client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            *history,
            {"role": "user", "content": message}
        ],
    )
    return response.choices[0].message.content

In [10]:
gr.ChatInterface(chatbot, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Implementing another llm for evaluating the chatbot

In [11]:
from pydantic import BaseModel


class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [12]:
evaluation_system_prompt = f"""You are an AI evaluator. Your task is to evaluate the responses of a chatbot based on the provided system prompt.
you deside if the response is acceptable or not. If it is not acceptable, provide feedback on how to improve it.
The Agent is playing the role of {name} and answering questions based on their LinkedIn profile and summary.`
The Agent has been instructed to be professional, as if talking to potential client of future employer.
The Agent has been instructed to provide accurate information based on the provided LinkedIn profile and summary.
The Linkedin profile is as follows:
{linkedin}
The summary is as follows:
{summary}
You will receive a message from the user and the response from the Agent. You will evaluate the response based on the following criteria:
1. Is the response professional and concise?
2. Does the response provide accurate information based on the provided LinkedIn profile and summary?
3. Does the response answer the user's question?
4. If the response is not acceptable, provide feedback on how to improve it.
"""

In [13]:
def evaluate_user_prompt(reply, message, history):
    user_prompt = f"Here is the conversation between the user and the Agent:\n\n{history}\n\n"
    user_prompt += f"Here is the latest message from the user:\n\n{message}\n\n"
    user_prompt += f"Here is the response from the Agent:\n\n{reply}\n\n"
    user_prompt += "Please evaluate the response based on the criteria provided in the system prompt."
    return user_prompt

In [14]:
google_client = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [15]:
def evaluate_chatbot(reply, message, history) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluation_system_prompt},
        {"role": "user", "content": evaluate_user_prompt(reply, message, history)}
    ]
    response = google_client.beta.chat.completions.parse(
        model="gemini-2.5-pro",
        messages=messages,
        response_format=Evaluation,
    )
    return response.choices[0].message.parsed

In [16]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is your experience with AI and FastAPI?"}
]
response = openai_client.chat.completions.create(
    model="gpt-5-mini",
    messages=messages,
)
reply = response.choices[0].message.content

In [17]:
reply

'Short answer: I use FastAPI regularly to build and deploy backend services — including AI inference endpoints — and I have hands-on experience building scalable, maintainable back-end components in Python.\n\nWhat I do with FastAPI and AI (high level)\n- Build REST/HTTP endpoints for model inference (sync and async), using Pydantic for request/response validation.\n- Wrap ML models (PyTorch/TensorFlow/transformers or custom inference code) behind lightweight FastAPI services for low-latency predictions.\n- Implement features common to AI services: batching, streaming responses, file uploads, background tasks for long-running jobs, and input validation.\n- Add production concerns: authentication, request rate limiting, logging/metrics, caching (to reduce redundant inference), and containerized deployments (Uvicorn/Gunicorn + Docker).\n- Optimize for performance: async handlers where applicable, efficient payload handling (JSON/NDJSON), and model-serve patterns that minimize cold starts

In [18]:
evaluate_chatbot(reply, "do you know about FastAPI?", messages[:1])

Evaluation(is_acceptable=False, feedback='The response is not acceptable because it fabricates a significant amount of information that is not present in the provided LinkedIn profile or summary. While FastAPI is listed as a top skill, the agent invents a detailed history of using it for AI inference endpoints, wrapping ML models, and other specific production concerns. The prompt explicitly instructs the agent to only provide accurate information from the source material and to say "I don\'t know" if the information isn\'t available. The agent should have simply confirmed that FastAPI is a top skill and perhaps related it to the general summary points about developing scalable back-end components, without inventing the entire AI specialization.')

In [19]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n ## Previous answer rejected\nYou just tried to reply, but the answer was rejected by the evaluator.\n\n"
    updated_system_prompt += f"Here is the response from the Agent:\n\n{reply}\n\n"
    updated_system_prompt += f"Here is the feedback from the evaluator:\n\n{feedback}\n\n"

    messages = openai_client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": updated_system_prompt},
            *history,
            {"role": "user", "content": message}
        ],
    )
    return messages.choices[0].message.content

In [22]:
def chatbot_with_evaluation(message, history):
    if not message.strip():
        return ""
    response_chat = openai_client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            *history,
            {"role": "user", "content": message}
        ],
    )
    reply_chatbot = response_chat.choices[0].message.content
    evaluation = evaluate_chatbot(reply_chatbot, message, history)
    if evaluation.is_acceptable:
        status = "Evaluation passed \n"
    else:
        status = f"Evaluation failed: \n{evaluation.feedback}"
        reply_chatbot = rerun(reply_chatbot, message, history, evaluation.feedback)
    return f"{status}\n\n{reply_chatbot}"

In [23]:
gr.ChatInterface(chatbot_with_evaluation, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
